In [6]:
from langchain.document_loaders import CSVLoader
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os
import pandas as pd
from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import time

# Load DATA

In [7]:
processed_file_path = "../Data/Process/sample_test_data.xlsx"
df_test = pd.read_excel(processed_file_path)

In [8]:
df_test.head()

,review,sentiment,cleaned_review
0,Yes bad acting is not only one thing to mentio...,negative,"['yes', 'bad', 'acting', 'not', 'thing', 'ment..."
1,Hard to imagine what they were thinking of whe...,negative,"['hard', 'imagine', 'think', 'movie', 'i.e.', ..."
2,This film is a nightmare The sensation you fee...,negative,"['film', 'nightmare', 'sensation', 'feel', 'wa..."
3,Duchess is a pretty white cat who lives with h...,positive,"['Duchess', 'pretty', 'white', 'cat', 'live', ..."
4,Like anyone else who bought this I was duped b...,negative,"['like', 'buy', 'dupe', 'piece', 'extreme', 'g..."


In [9]:
# # Load environment variables from .env file
# load_dotenv()
# # Access the API key
# google_palm_api_key = os.getenv('Google_palm_api_key')
# llm = GooglePalm(google_api_key=api_key, temperature=0.4)
# # classification_chain = load_qa_chain(llm=llm)



# def get_sentiment(text):
#     print(text)
#     # Define the prompt template
#     prompt_template = """Given the review, ask as the NLP model to determine the sentiment of the below review: "{text}"."""

#     # Generate the complete prompt by inserting the review text
#     prompt = prompt_template.format(text=text)

#     # Pass the prompt to the LLM
#     classification = llm(prompt)
    
#     return classification

# # df_test['pred_review'] = df_test['review'].apply(lambda x: get_sentiment(x))

In [10]:
df_test['len_review'] = df_test['review'].apply(lambda x: len(x.split()))

In [11]:
df_test['len_review'].mean()

232.616

In [12]:
df_test.shape

(1000, 4)

In [13]:
# Access the Hugging Face API key
huggingface_api_key = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# Initialize the LLM with Hugging Face
llm = HuggingFaceHub(
    huggingfacehub_api_token=huggingface_api_key,
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 0.1},
    task="text2text-generation"
)

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["text"],
    template="Given the review, determine the sentiment of the below review: {text}"
)

# Create the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt_template
) 

# Function to classify sentiment
def classify_review(review):
    result = chain.run(text=review)
    return result.strip().lower()


accuracy_list = []

for i in range(0, len(df_test), 10):
    batch_df = df_test.iloc[i:i+10]
    
    batch_df.loc['predicted_sentiment'] = batch_df['review'].apply(classify_review)
    
    batch_accuracy = batch_df[batch_df['sentiment'] == batch_df['predicted_sentiment']].shape[0] / batch_df.shape[0]
    accuracy_list.append(batch_accuracy)
    
    print(f"Batch {i//10 + 1} accuracy: {batch_accuracy}")
    
    # Pause for a few seconds to avoid HTTP limit errors
    time.sleep(2)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Overall accuracy: {overall_accuracy}")

C:\Users\ravip\AppData\Local\Temp\ipykernel_15192\3064228076.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['predicted_sentiment'] = batch_df['review'].apply(classify_review)


Batch 1 accuracy: 1.0


C:\Users\ravip\AppData\Local\Temp\ipykernel_15192\3064228076.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['predicted_sentiment'] = batch_df['review'].apply(classify_review)


Batch 2 accuracy: 1.0


C:\Users\ravip\AppData\Local\Temp\ipykernel_15192\3064228076.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['predicted_sentiment'] = batch_df['review'].apply(classify_review)


Batch 3 accuracy: 1.0


KeyboardInterrupt: 

In [ ]:
accuracy = df_test[df_test['sentiment']==df_test['predicted_sentiment']].shape[0]/df_test.shape[0]

In [ ]:
result = chain.run(text=review)